In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=False)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from os import listdir
import os
from pickle import dump
from pickle import load

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

import string
import re

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
project =list

In [0]:
# extract features from each photo in the directory
def extract_features(directory):
	# load the model - Oxford Visual Geometry Group
	model = VGG16()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# summarize
	# print(model.summary())
	# extract features from each photo
	features = dict()
	for name in listdir(directory):
		# load an image from file
		filename = directory + '/' + name
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		print('>%s' % name)
	return features

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = np.zeros(shape=(1,4097))

In [0]:
# extract features from all images
from os import listdir
from os.path import isfile, join

# Setting flag = false to skip the first path as it is the path of the root itself and will go infite.
flag = False 

mypath = "/content/drive/My Drive/Database"
# onlyfiles = [f for f in os.walk(mypath) if isfile(join(mypath, f))]

for path, dirs, files in os.walk(mypath):
     directory = path
     if flag == True:
        features = extract_features(directory)
        keys =features.keys()
        for key in keys:
          val= (features.get(key))
          val = np.array(val)
          labelname = np.array(os.path.basename(os.path.normpath(directory)))
          val =np.append(val,labelname)
          data =np.row_stack((data,val))
     else:
       flag =True
# directory = '/content/drive/My Drive/Colab Notebooks/01'
# features, project = extract_features(directory)
# print('Extracted Features: %d' % len(features))
# # save to file
# dump(features, open('features.pkl', 'wb'))












>00005_930831_fa.ppm.bz2.ppm
>00005_930831_fb.ppm.bz2.ppm
>00005_930831_fb.ppm.bz2.png
>00005_930831_fa.ppm.bz2.png
>00004_930831_fa.ppm
>00004_930831_fb.ppm
>00004_930831_fb.ppm.bz2.png
>00004_930831_fa.ppm.bz2.png
>00008_930831_fa_a.ppm.bz2.ppm
>00008_930831_fb_a.ppm.bz2.ppm
>00008_930831_fa_a.ppm.bz2.png
>00008_930831_fb_a.ppm.bz2.png
>00006_930831_fa_a.ppm.bz2.ppm
>00006_930831_fb_a.ppm.bz2.ppm
>00006_930831_fb_a.ppm.bz2.png
>00006_930831_fa_a.ppm.bz2.png
>00007_930831_fa_a.ppm.bz2.ppm
>00007_930831_fb_a.ppm.bz2.ppm
>00007_930831_fa_a.ppm.bz2.png
>00007_930831_fb_a.ppm.bz2.png


In [0]:
# from PIL import Image
# import numpy as np

# w, h = 512, 512
# data = np.zeros((h, w, 3), dtype=np.uint8)
# data[256, 256] = [255, 0, 0]
# img = Image.fromarray(project, 'RGB')
# img.save('my.png')
# img.show()

In [0]:
# keys =features.keys()
# for key in keys:
#   val= (features.get(key))
#   data =np.row_stack((data,val))

In [0]:
data = data[1:]
data = pd.DataFrame(data)
data

In [0]:
dump(data, open('features.pkl', 'wb'))

In [0]:
import pickle
pickle_in = open("features.pkl","rb")
df = pickle.load(pickle_in)

In [0]:
data = pd.DataFrame(data)

In [0]:
columns =data.columns

In [0]:
data.shape

In [0]:
persons = data[columns[-1]].unique()

In [0]:

# new_data = np.linspace(1,4098, num=4098)
# new_data= np.reshape(new_data, (1,4098))

In [0]:
new_data = np.zeros(shape = (1,data.shape[1]+1))
# new_data = new_data.reshape(-1,1)
new_data.shape

In [0]:
for p in persons:
  d = data.loc[data[columns[-1]] ==  p]
  d = np.array(d, dtype =float)
  d = np.absolute(d[0,:-1]-d[1,:-1])
  d = np.append(d,[p,-1])
  new_data = np.row_stack((d,new_data))
new_data = new_data[:-1,:]  


In [0]:
# new_data[0] = np.append(new_data,4099)

In [0]:
new_data = pd.DataFrame(new_data)
# new_header = new_data.iloc[0] #grab the first row for the header
# new_data = new_data[1:] #take the data less the header row
# new_data.columns = new_header #set the header row as the df header
person = list(new_data[columns[-1]])
person = np.array(person)
new_data['Person'] = np.array(person, dtype=float)


In [0]:
label = np.array(list(new_data[new_data.columns[len(new_data.columns)-2]]))
new_data['Label'] = np.array(label, dtype=float)


In [0]:
new_data.columns = np.array(new_data.columns)

In [0]:
new_data = new_data.drop([new_data.columns[len(new_data.columns)-3]],axis=1)

In [0]:
new_data =new_data.drop([new_data.columns[len(new_data.columns)-3]],axis=1)

In [0]:
new_datac2 = np.zeros(shape = (1,data.shape[1]+1))
np.random.seed(0)

In [0]:
for i in range(persons.shape[0]):
    for j in range(persons.shape[0]):
        if i != j:
            p1 = data.loc[data[columns[-1]] ==  persons[i]]
            p2 = data.loc[data[columns[-1]] ==  persons[j]]
            
            p1 = np.array(p1, dtype= float)
            p2 = np.array(p2, dtype = float)
            p3 = np.row_stack((p2[1],p2[0]))
            # p3 = np.array(p3, dtype=float)
            p = np.absolute(p1-p2)
            k = np.absolute(p1-p3)
            # label  = (((int(persons[i]))+1)*100)+((int(persons[j]))+1)
            # label = np.array([label,label])
            # label = label.T
            c2 = np.array([1,1])
            c2= c2.T
            # p = np.column_stack((p,label))
            p = np.column_stack((p,c2))
            # k = np.column_stack((k,label))
            k = np.column_stack((k,c2))
            new_datac2 = np.row_stack((p,new_datac2))
            new_datac2 = np.row_stack((k,new_datac2))          


In [0]:
new_datac2 = new_datac2[:-1]
new_datac2 = pd.DataFrame(new_datac2)


In [0]:
new_datac2['Person'] = np.array(new_datac2.iloc[:,4096])


In [0]:
new_datac2['Label'] = np.array(new_datac2.iloc[:,4097])

In [0]:
new_datac2 = new_datac2.drop([new_datac2.columns[len(new_datac2.columns)-3]],axis=1)


In [0]:
new_datac2 = new_datac2.drop([new_datac2.columns[len(new_datac2.columns)-3]],axis=1)

In [0]:
ndata = pd.concat([new_data,new_datac2])

In [0]:
ndata

In [0]:
ndata.loc[ndata['Label']==0]

In [0]:
training = ndata
training = training.drop(['Person'],axis=1)
training = np.array(training,dtype=float)
x_train = training[:,:-1]
y_train = training[:,-1]
from sklearn.svm import SVC
clf = SVC(gamma='auto',kernel='linear')
clf.fit(x_train,y_train)

print (len(clf.support_vectors_))
print (clf.n_support_)
clf.score(x_train,y_train)
print (np.abs(clf.dual_coef_))

## Setting Global values for aplha,intercept,and others


In [0]:
alphas = clf.dual_coef_
support_vectors = clf.support_vectors_
bias = clf.intercept_
Ns = len(support_vectors)
print ('Aplhas',alphas)
print ('Support Vectors',support_vectors)
print ('Bias',bias)
print ('Ns', Ns)

In [0]:
# Creating test set
testSet = extract_features("/content/drive/My Drive/TestDatabase/00020")

In [0]:
input = np.array(list(testSet.values())[0][0])

### Verification Part


In [0]:
# Helper function
def getX(ci):
  res = np.array(data.loc[data[data.columns[len(data.columns)-1]] == ci], dtype= float)
  # print ("Delta", res[0]-res[1])
  return res[0,:-1]


In [0]:
getX("00005")

In [0]:
def subtract(a,b):
   a= np.array(a, dtype =float)
   b = np.array(b,dtype = float)
   print ("a",a.shape)
   print ("b",b.shape)
   return np.absolute(a- b)

In [0]:
def computeKernal(a,sub):
  sub = sub.reshape(-1,1)
  return (a) @ (sub)

In [0]:
'''
Terminologies:
1. probe = test image
2. ci = claimed identity
'''
def identify(probe,ci):
    # x_ = extract_proble_features(probe)
    x = getX(ci)
    sub = subtract(x,probe)
    similarity = 0
    for i in range(Ns):
         alpha = alphas[0][i]
         similarity = similarity + (alpha * computeKernal(support_vectors[i],sub))

    # print ("Similarity",similarity)
    return similarity
     

In [0]:
ans =list()
identify(input,"00006")

In [0]:
res = np.array(data.loc[data[data.columns[len(data.columns)-1]] == "00005"], dtype= float)
identify(res[1,:-1],"00005")